In [2]:
from datascience import *
import numpy as np
import matplotlib
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

### Multiple regression

We'll illustrate multiple linear regression using a toy "Zillow competition" problem, using a data set of house sales in Ames, Iowa. This example parallels 
the treatment in [Chapter 17.6](https://www.inferentialthinking.com/chapters/17/6/Multiple_Regression.html) of the book.


In [3]:
# Let's just get data from 1 Family houses and a subset of the columns
all_sales = Table.read_table('house.csv')
sales = all_sales.where('Bldg Type', '1Fam').where('Sale Condition', 'Normal').select(
    'SalePrice', '1st Flr SF', '2nd Flr SF', 
    'Total Bsmt SF', 'Garage Area', 
    'Wood Deck SF', 'Open Porch SF', 'Lot Area', 
    'Year Built', 'Yr Sold')
sales.sort('SalePrice')

SalePrice | 1st Flr SF | 2nd Flr SF | Total Bsmt SF | Garage Area | Wood Deck SF | Open Porch SF | Lot Area | Year Built | Yr Sold
35000     | 498        | 0          | 498           | 216         | 0            | 0             | 8088     | 1922       | 2006
39300     | 334        | 0          | 0             | 0           | 0            | 0             | 5000     | 1946       | 2007
40000     | 649        | 668        | 649           | 250         | 0            | 54            | 8500     | 1920       | 2008
45000     | 612        | 0          | 0             | 308         | 0            | 0             | 5925     | 1940       | 2009
52000     | 729        | 0          | 270           | 0           | 0            | 0             | 4130     | 1935       | 2008
52500     | 693        | 0          | 693           | 0           | 0            | 20            | 4118     | 1941       | 2006
55000     | 723        | 363        | 723           | 400         | 0            | 24            | 11340    | 1920       | 2008
55000     | 796        | 0          | 796           | 0           | 0            | 0             | 3636     | 1922       | 2008
57625     | 810        | 0          | 0             | 280         | 119          | 24            | 21780    | 1910       | 2009
58500     | 864        | 0          | 864           | 200         | 0            | 0             | 8212     | 1914       | 2010
... (1992 rows omitted)

In [4]:
# View a histogram of the sale prices


In [5]:
# View a scatter plot looking at the first floor square footage and price


In [6]:
# Defining our function to get the correlation
def standardize(x):
    return (x-np.mean(x))/np.std(x)

def correlation(t, col1, col2):
    x = standardize(t.column(col1))
    y = standardize(t.column(col2))
    return np.mean(x*y)


In [7]:
# Look at the correlation between sales price and first floor square footage


In [8]:
# Get the correlation between all our variables and price
r = make_array()
for label in sales.labels:
    r = np.append(r, correlation(sales, label, 'SalePrice'))

corr_table = Table().with_columns('Attribute', sales.labels, 'Correlation with Sale Price', r)
corr_table

Attribute     | Correlation with Sale Price
SalePrice     | 1
1st Flr SF    | 0.642466
2nd Flr SF    | 0.357522
Total Bsmt SF | 0.652979
Garage Area   | 0.638594
Wood Deck SF  | 0.352699
Open Porch SF | 0.336909
Lot Area      | 0.290823
Year Built    | 0.565165
Yr Sold       | 0.0259486

In [9]:
# Look at the correlation between first and second floor square footage
# Any explanation for this relationship? 



In [10]:
# Relationship between square footage of both floors and price



In [11]:
# Add a column of all ones to the data
# This will enable us to have an intercept in our regression model



In [ ]:
# continue from here - some choices that need to be made about what to cut, etc. 

In [14]:
# Create training and test splits of our data using the tb.split() method



1001 training and 1001 test instances.


In [16]:
# Create a function that takes a row and predicts the price based on a vector of coefficents called "slopes"
def predict(slopes, row):
    
    

In [17]:
# Use row 0 as an example to make predictions on


# Create random slopes (this should lead to poor predictions)


# The resulting predicted price based on the random slopes



In [18]:
# Comparing the actual price and predicted price



In [ ]:
# create a function that returns the RMSE for given slopes, features and actual prices
def rmse(slopes, attributes, prices):
    
    
    
    
    

In [19]:
# Write a function to get the RMSE for the training data using our randomly generated slopes






In [21]:
# Get the "best slopes" by miniming the RMSE



In [22]:
# Show the best slopes found and the RMSE found on the training data


# Can you write out the regression equation? 



In [24]:
# Let's now evaluate the RMSE on the test data





In [ ]:
# A function that produces predicted values using the "best slopes" found by minimizing the RMSE
# We will use this function to make 
def fit(row):




In [ ]:
# Add the predicted values to the test set, and visualize how close the predictions are




In [ ]:
# Create a residual plot to see if there are patterns in the data that are unexplained



In [26]:
# Let's try to use nearest neighbor regression rather than a linear classifier...




In [27]:
# Create the functions we need for our nearest neighbor classifier...

def distance(pt1, pt2):
    """The distance between two points, represented as arrays."""
    return np.sqrt(sum((pt1 - pt2) ** 2))

def row_distance(row1, row2):
    """The distance between two rows of a table."""
    return distance(np.array(row1), np.array(row2))

def distances(training, example, output):
    """Compute the distance from example for each row in training."""
    dists = []
    attributes = training.drop(output)
    for row in attributes.rows:
        dists.append(row_distance(row, example))
    return training.with_column('Distance', dists)

def closest(training, example, k, output):
    """Return a table of the k closest neighbors to example."""
    return distances(training, example, output).sort('Distance').take(np.arange(k))


In [28]:
# Let's get the 5 closest houses to the home in the first row of the test set




In [29]:
# Let's create a prediction function that returns the average house value of the most similar houses
def predict_nn(example):
    """Return the majority class among the k nearest neighbors."""

    


In [ ]:
# Print the actual and predicted home price



In [ ]:
# Let's calculate the RMSE on the test set for the kNN classifier
nn_test_predictions = test_nn.drop('SalePrice').apply(predict_nn)
rmse_nn = np.mean((test_prices - nn_test_predictions) ** 2) ** 0.5

# Which is better, linear regression of kNN? 




In [ ]:
# Let's look at the pattern of residuals again

